In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

### 데이터 수집

In [64]:
train = pd.read_csv('./kaggle_data/Train.csv',index_col='ID')
test = pd.read_csv('./kaggle_data/test.csv',index_col='ID')

In [65]:
train

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,
1,C,Ship,2.0,2,135,3.0,low,M,NaN,1339,1
2,F,Flight,5.0,3,225,5.0,low,F,6.0,1082,0
3,F,Ship,3.0,1,229,3.0,low,M,NaN,4971,1
4,F,Ship,3.0,2,228,NaN,medium,M,2.0,5640,0
5,D,Flight,NaN,1,195,6.0,high,F,NaN,4944,1
...,...,...,...,...,...,...,...,...,...,...,...
6995,D,Ship,4.0,2,232,5.0,medium,F,4.0,1783,0
6996,F,Ship,3.0,5,228,2.0,medium,F,NaN,5208,0
6997,D,Ship,NaN,1,300,5.0,low,F,10.0,1787,0


In [66]:
test

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
ID,,,,,,,,,,
7000,F,Ship,3.0,1,274,NaN,high,M,NaN,4352
7001,F,Ship,3.0,4,136,2.0,medium,F,NaN,1056
7002,A,Flight,NaN,5,140,3.0,low,F,7.0,5383
7003,C,Ship,NaN,1,291,4.0,low,F,NaN,1880
7004,F,?,4.0,2,147,3.0,low,F,5.0,5174
...,...,...,...,...,...,...,...,...,...,...
10995,A,?,4.0,1,204,4.0,?,F,NaN,1667
10996,C,Ship,NaN,3,195,2.0,medium,M,NaN,3869
10997,B,Flight,NaN,3,206,2.0,medium,M,7.0,4531


### 컬럼 정리
- ID: 고유 식별자
- Warehouse_block: 창고 블록
- Mode_of_Shipment: 운송 방식
- Customer_care_calls: 고객 서비스 호출 수
- Customer_rating: 고객 평점
- Cost_of_the_Product: 제품 비용
- Prior_purchases: 이전 구매 횟수
- Product_importance: 제품 중요도
- Gender: 성별
- Discount_offered: 제공된 할인
- Weight_in_gms: 무게 (그램 단위)
- Reached.on.Time_Y.N: 제 시간에 도달했는지 여부 (예상되는 예측 변수)

In [71]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999 entries, 1 to 6999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  5423 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6049 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Gender               6999 non-null   object 
 8   Discount_offered     3468 non-null   float64
 9   Weight_in_gms        6999 non-null   object 
 10  Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 656.2+ KB


In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 7000 to 10999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      4000 non-null   object 
 1   Mode_of_Shipment     4000 non-null   object 
 2   Customer_care_calls  3144 non-null   float64
 3   Customer_rating      4000 non-null   int64  
 4   Cost_of_the_Product  4000 non-null   int64  
 5   Prior_purchases      3633 non-null   float64
 6   Product_importance   4000 non-null   object 
 7   Gender               4000 non-null   object 
 8   Discount_offered     2085 non-null   float64
 9   Weight_in_gms        4000 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 343.8+ KB


### 데이터 전처리
- train 결측치 컬럼: Customer_care_calls, Prior_purchases, Discount_offered
- test 결측치 컬럼 : Customer_care_calls, Prior_purchases, Discount_offered

In [67]:
train.corr()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Reached.on.Time_Y.N
Customer_care_calls,1.000000,0.013420,0.047041,0.173709,-0.158465,-0.070266
Customer_rating,0.013420,1.000000,-0.001961,0.008217,-0.016456,0.011275
Cost_of_the_Product,0.047041,-0.001961,1.000000,0.028947,-0.139349,-0.012811
Prior_purchases,0.173709,0.008217,0.028947,1.000000,-0.081868,-0.065160
Discount_offered,-0.158465,-0.016456,-0.139349,-0.081868,1.000000,0.397393
Reached.on.Time_Y.N,-0.070266,0.011275,-0.012811,-0.065160,0.397393,1.000000


- 낮은 상관관계를 가지는 것으로 보인다.

In [54]:
X_train= train.drop('Reached.on.Time_Y.N', axis=1)
X_test= test
y_train=train['Reached.on.Time_Y.N']

In [55]:
X_train.shape, y_train.shape, X_test.shape

((6999, 10), (6999,), (4000, 10))

In [25]:
# 인코딩이 필요한 컬럼 추출
cat_feature=['Warehouse_block','Mode_of_Shipment','Product_importance','Weight_in_gms']

In [18]:
#원핫인코딩
for cat_name in cat_feature:
    dummy = pd.get_dummies(X_train[cat_name], prefix=cat_name) # prefix: 접두사
    X_train = pd.concat([X_train, dummy],axis=1)
    X_train.drop(cat_name, axis=1, inplace=True)

NameError: name 'cat_feature' is not defined

### 모델 학습

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tree = DecisionTreeClassifier()

In [ ]:
param_tree = {
    'max_depth' : range(1,6),
    'min_samples_split':range(1, 101, 10),
    'min_samples_leaf': range(1, 101, 10),
    'max_leaf_nodes':range(1, 51, 10)
}

In [ ]:
grid_search_tree = GridSearchCV(tree, param_tree, cv=5)
grid_search_tree

In [ ]:
grid_search_tree.fit(X_train, y_train)

In [ ]:
grid_search_tree.best_params_

In [ ]:
# 답안지 작성
omr = pd.read_csv('./data/gender_submission.csv')
omr['Survived'] = pre
omr.to_csv('ada_ta01.csv',index=False)